In [1]:
import pymssql
import pandas as pd
import importlib #提供import语句
import sys
import time #提供延时功能
from selenium.webdriver.common.by import By    #用于获取网页中的相关元素、标签
from selenium import webdriver
importlib.reload(sys)    #避免utf-8等编码问题的出现

<module 'sys' (built-in)>

In [2]:
class aqc():
    def __init__(self):
        pass

    # 数据库获取公司名称
    def company(self):
        conn = pymssql.connect(host="10.2.0.45:1433", user="sa", password="Frbi2020", database="CyBi", charset="utf8")
        df = pd.read_sql("select * from dbo.View_Frbi_gsxx order by 数量 desc",con=conn)
        conn.close()
        return df['客户']
    
    # 模拟打开浏览器并完成登录
    def browser_login(self):
        #伪装成浏览器，防止被识破
        option = webdriver.ChromeOptions()
        option.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"')    #请求头
        driver = webdriver.Chrome(options=option)    #打开chrome浏览器
        #打开爱企查登录页面
        driver.get('https://aiqicha.baidu.com/')
        time.sleep(30) #等待20s以用于完成手动登录操作
        return driver
    
    # 初次搜索目标
    def search_first(self, driver, txt):
        # 主页面搜索
        driver.find_element(By.ID,'aqc-search-input').send_keys(txt)
        srh_btn = driver.find_element(By.XPATH,'/html/body/div/div[2]/div/div[1]/div[2]/button')
        srh_btn.click()
        time.sleep(3)
        # 列表窗口点击进入详情窗口
        srh_a = driver.find_element(By.XPATH,'/html/body/div/div[2]/div/div[1]/div[@class="list-wrap"]/div[2]/div/div[1]/div[2]/div/h3/a')
        srh_a.click()
        time.sleep(5)
        # 关闭列表窗口并切换至详情窗口
        driver.close()
        ws=driver.window_handles[0]
        driver.switch_to.window(ws)
    
    # 持续搜索目标
    def search(self, driver, txt):
        # 清楚搜索框数据，并搜索新内容
        driver.find_element(By.ID,'aqc-header-search-input').clear()
        driver.find_element(By.ID,'aqc-header-search-input').send_keys(txt)
        srh_btn = driver.find_element(By.XPATH,'/html/body/div/div[1]/header/div/div[2]/button')
        srh_btn.click()
        time.sleep(3)
        # 列表窗口点击进入详情窗口
        srh_a = driver.find_element(By.XPATH,'/html/body/div/div[2]/div/div[1]/div[@class="list-wrap"]/div[2]/div/div[1]/div[2]/div/h3/a')
        srh_a.click()
        time.sleep(5)
        # 关闭列表窗口并切换至详情窗口
        driver.close()
        ws=driver.window_handles[0]
        driver.switch_to.window(ws)

    # 获取公司信息
    def get_res(self, driver, txt):
        # 通用链接
        universal_link = '/html/body/div[1]/div[2]/div/div[6]/div[1]/div[3]/table/tbody'
        try:
            # 名称
            name = driver.find_element(By.XPATH, universal_link + '/tr[1]/td[2]/span').text
        except:
            name = ''
        try:
            # 电话
            phone = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div[5]/div[1]/div[1]/div/span[2]/span/span').text
        except:
            phone = ''
        try:
            # 邮箱
            email = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div[2]/div[1]/div[1]/div[2]/div[5]/div[1]/div[2]/div/a').text
        except:
            email = ''
        try:
            # 统一社会信用代码
            code = driver.find_element(By.XPATH, universal_link + '/tr[1]/td[4]/span').text
        except:
            code = ''
        try:
            # 法定代表人
            representative = driver.find_element(By.XPATH, universal_link + '/tr[2]/td[2]/div[2]/a').text
        except:
            representative = ''
        try:
            # 经营状态
            status = driver.find_element(By.XPATH, universal_link + '/tr[2]/td[4]').text
        except:
            status = ''
        try:
            # 成立日期
            establish = driver.find_element(By.XPATH, universal_link + '/tr[3]/td[2]').text
        except:
            establish = ''
        try:
            # 行政区划
            district = driver.find_element(By.XPATH, universal_link + '/tr[3]/td[4]').text
        except:
            district = ''
        try:
            # 注册资本
            capital = driver.find_element(By.XPATH, universal_link + '/tr[4]/td[2]').text
        except:
            capital = ''
        try:
            # 企业类型
            type = driver.find_element(By.XPATH, universal_link + '/tr[5]/td[2]').text
        except:
            type = ''
        try:
            # 所属行业
            industry = driver.find_element(By.XPATH, universal_link + '/tr[5]/td[4]').text
        except:
            industry = ''
        try:
            # 营业期限
            term = driver.find_element(By.XPATH, universal_link + '/tr[8]/td[2]').text
            term_start = term[:10]
            term_end = term[-10:]
        except:
            term_start = ''
            term_end = ''
        try:
            # 参保人数
            insured_persons = driver.find_element(By.XPATH, universal_link + '/tr[9]/td[4]').text
        except:
            insured_persons = ''
        try:
            # 曾用名
            former = driver.find_element(By.XPATH, universal_link + '/tr[10]/td[2]/p/span').text
        except:
            former = ''
        try:
            # 注册地址
            address = driver.find_element(By.XPATH, universal_link + '/tr[11]/td[2]/span').text
        except:
            address = ''
        try:
            # 经营范围
            scope = driver.find_element(By.XPATH, universal_link + '/tr[12]/td[2]/div').text.replace('\n... 展开','')
        except:
            scope = ''

        res = [txt, name, phone, email, code, representative, status, establish, district, capital, type, industry, term_start, term_end, insured_persons, former, address, scope]
        for j in range(len(res)):
            res[j] = str(res[j])
        return res

In [3]:
aq = aqc()
company = list(aq.company())

d:\Digital\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
driver = aq.browser_login()
resd = []
for i in range(len(company)):
    txt = company[i]
    if i == 0:
        try:
            aq.search_first(driver, txt)
        except (Exception, BaseException) as e:
            print(e)
            time.sleep(10)
            continue
    else:
        try:
            aq.search(driver, txt)
        except (Exception, BaseException) as e:
            print(e)
            time.sleep(10)
            continue
    res = aq.get_res(driver, txt)
    print(res)
    resd.append(res)
resd = pd.DataFrame(resd)
resd.columns = ['txt', 'name', 'phone', 'email', 'code', 'representative', 'status', 'establish', 'dsitrict', 'capital', 'type', 'industry', 'term_start', 'term-end', 'insured_persons', 'former', 'address', 'scope']
print(resd)
resd.to_csv("gsxx.csv",mode='a',index=False,header=True)
driver.close()

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div/div[2]/div/div[1]/div[@class="list-wrap"]/div[2]/div/div[1]/div[2]/div/h3/a"}
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F0A813+48355]
	(No symbol) [0x00E9C4B1]
	(No symbol) [0x00DA5358]
	(No symbol) [0x00DD09A5]
	(No symbol) [0x00DD0B3B]
	(No symbol) [0x00DFE232]
	(No symbol) [0x00DEA784]
	(No symbol) [0x00DFC922]
	(No symbol) [0x00DEA536]
	(No symbol) [0x00DC82DC]
	(No symbol) [0x00DC93DD]
	GetHandleVerifier [0x0116AABD+2539405]
	GetHandleVerifier [0x011AA78F+2800735]
	GetHandleVerifier [0x011A456C+2775612]
	GetHandleVerifier [0x00F951E0+616112]
	(No symbol) [0x00EA5F8C]
	(No symbol) [0x00EA2328]
	(No symbol) [0x00EA240B]
	(No symbol) [0x00E94FF7]
	BaseThreadInitThunk [0x772D6359+25]
	RtlGetAppContainerNamedObjectPath [0x77D77B74+228]
	RtlGetAppContainerNamedObjectPath [0x77D77B44+180]

Message: no such element: Unable to locate element: {"metho

KeyboardInterrupt: 

In [7]:
resd = pd.DataFrame(resd)
resd.columns = ['txt', 'name', 'phone', 'email', 'code', 'representative', 'status', 'establish', 'dsitrict', 'capital', 'type', 'industry', 'term_start', 'term-end', 'insured_persons', 'former', 'address', 'scope']
print(resd)
resd.to_csv("gsxx.csv",mode='a',index=False,header=True)

                txt            name         phone                email  \
0      鲁西化工集团股份有限公司    鲁西化工集团股份有限公司  0635-3481198      000830@lxhg.com   
1    中铁一局集团新运工程有限公司  中铁一局集团新运工程有限公司   15922909993                        
2      杭州赢奥机械设备有限公司    杭州赢奥机械设备有限公司   17354734335   17354734335@qq.com   
3     河津市海军机械工贸有限公司   河津市海军机械工贸有限公司   13934100961  13934100961@163.com   
4      介休安益新昌煤化有限公司    介休安益新昌煤化有限公司   13753445758    1311593389@qq.com   
..              ...             ...           ...                  ...   
185        景洪市交通运输局        景洪市交通运输局  0691-2123673                        
186      绿洲森工股份有限公司                       2761353      ahlz008@163.com   
187  北京北方中益重型机械有限公司  北京北方中益重型机械有限公司  010-87744776     314169144@qq.com   
188  大连沃乐建筑工程机械有限公司  大连沃乐建筑工程机械有限公司   13942080999     154108741@qq.com   
189  重庆市开州区科鑫实业有限公司                                                      

                   code representative status   establish         dsitrict  \
0    91370000614071479T          

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F0A813+48355]
	(No symbol) [0x00E9C4B1]
	(No symbol) [0x00DA5358]
	(No symbol) [0x00D970EB]
	(No symbol) [0x00D977AA]
	(No symbol) [0x00D97748]
	(No symbol) [0x00D8C2AA]
	(No symbol) [0x00D8CB1D]
	(No symbol) [0x00DF27B8]
	(No symbol) [0x00DEA703]
	(No symbol) [0x00DC82DC]
	(No symbol) [0x00DC93DD]
	GetHandleVerifier [0x0116AABD+2539405]
	GetHandleVerifier [0x011AA78F+2800735]
	GetHandleVerifier [0x011A456C+2775612]
	GetHandleVerifier [0x00F951E0+616112]
	(No symbol) [0x00EA5F8C]
	(No symbol) [0x00EA2328]
	(No symbol) [0x00EA240B]
	(No symbol) [0x00E94FF7]
	BaseThreadInitThunk [0x772D6359+25]
	RtlGetAppContainerNamedObjectPath [0x77D77B74+228]
	RtlGetAppContainerNamedObjectPath [0x77D77B44+180]


In [8]:
resd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   txt              190 non-null    object
 1   name             190 non-null    object
 2   phone            190 non-null    object
 3   email            190 non-null    object
 4   code             190 non-null    object
 5   representative   190 non-null    object
 6   status           190 non-null    object
 7   establish        190 non-null    object
 8   dsitrict         190 non-null    object
 9   capital          190 non-null    object
 10  type             190 non-null    object
 11  industry         190 non-null    object
 12  term_start       190 non-null    object
 13  term-end         190 non-null    object
 14  insured_persons  190 non-null    object
 15  former           190 non-null    object
 16  address          190 non-null    object
 17  scope            190 non-null    ob